Load Data

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import email

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!unzip gdrive/My\ Drive/emails.csv.zip

unzip:  cannot find or open gdrive/My Drive/emails.csv.zip, gdrive/My Drive/emails.csv.zip.zip or gdrive/My Drive/emails.csv.zip.ZIP.


In [ ]:
!unzip gdrive/My\ Drive/emails.csv.zip > /dev/null

unzip:  cannot find or open gdrive/My Drive/emails.csv.zip, gdrive/My Drive/emails.csv.zip.zip or gdrive/My Drive/emails.csv.zip.ZIP.


In [ ]:
# considering the run-time, we only read the first 35000 rows here
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/emails.csv", nrows = 1000)

df.shape

In [ ]:
df.head()

,file,message
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...


In [ ]:
df.describe()

,file,message
count,35000,35000
unique,35000,35000
top,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...
freq,1,1


In [ ]:
email_sent = df[df["file"].str.contains('sent').tolist()]

In [ ]:
email_sent = email_sent.assign(sender=email_sent["file"].map(lambda x: re.search("(.*)/.*sent", x).group(1)).values)
email_sent.drop("file", axis=1, inplace=True)
email_sent["sender"].value_counts().head(10)

Extract Raw Text

In [ ]:
emails = list(map(email.parser.Parser().parsestr,df['message']))
# extract headings
# From
# To
# ...
headings  = emails[0].keys()
# extract the useful info for headings
for key in headings:
    df[key] = [doc[key] for doc in emails]

def raw_text(emails):
    text = []
    for email in emails.walk():
        if email.get_content_type() == 'text/plain':
            text.append(email.get_payload())
    return ''.join(text)

# extract text body from raw text
df['body'] = list(map(raw_text, emails))
# extract the user from file
df['user'] = df['file'].map(lambda x: x.split('/')[0])

df.head()

,file,message,Message-ID,Date,From,To,Subject,Mime-Version,Content-Type,Content-Transfer-Encoding,X-From,X-To,X-cc,X-bcc,X-Folder,X-Origin,X-FileName,body,user
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...,<18782981.1075855378110.JavaMail.evans@thyme>,"Mon, 14 May 2001 16:39:00 -0700 (PDT)",phillip.allen@enron.com,tim.belden@enron.com,,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,Tim Belden <Tim Belden/Enron@EnronXGate>,,,"\Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Se...",Allen-P,pallen (Non-Privileged).pst,Here is our forecast\n\n,allen-p
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...,<15464986.1075855378456.JavaMail.evans@thyme>,"Fri, 4 May 2001 13:51:00 -0700 (PDT)",phillip.allen@enron.com,john.lavorato@enron.com,Re:,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,John J Lavorato <John J Lavorato/ENRON@enronXg...,,,"\Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Se...",Allen-P,pallen (Non-Privileged).pst,Traveling to have a business meeting takes the...,allen-p
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...,<24216240.1075855687451.JavaMail.evans@thyme>,"Wed, 18 Oct 2000 03:00:00 -0700 (PDT)",phillip.allen@enron.com,leah.arsdall@enron.com,Re: test,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,Leah Van Arsdall,,,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Allen-P,pallen.nsf,test successful. way to go!!!,allen-p
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...,<13505866.1075863688222.JavaMail.evans@thyme>,"Mon, 23 Oct 2000 06:13:00 -0700 (PDT)",phillip.allen@enron.com,randall.gay@enron.com,,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,Randall L Gay,,,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Allen-P,pallen.nsf,"Randy,\n\n Can you send me a schedule of the s...",allen-p
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...,<30922949.1075863688243.JavaMail.evans@thyme>,"Thu, 31 Aug 2000 05:07:00 -0700 (PDT)",phillip.allen@enron.com,greg.piper@enron.com,Re: Hello,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,Greg Piper,,,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Allen-P,pallen.nsf,Let's shoot for Tuesday at 11:45.,allen-p


In [ ]:
df.dtypes

file                         object
message                      object
Message-ID                   object
Date                         object
From                         object
To                           object
Subject                      object
Mime-Version                 object
Content-Type                 object
Content-Transfer-Encoding    object
X-From                       object
X-To                         object
X-cc                         object
X-bcc                        object
X-Folder                     object
X-Origin                     object
X-FileName                   object
body                         object
user                         object
dtype: object

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35000 entries, 0 to 34999
Data columns (total 19 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   file                       35000 non-null  object
 1   message                    35000 non-null  object
 2   Message-ID                 35000 non-null  object
 3   Date                       35000 non-null  object
 4   From                       35000 non-null  object
 5   To                         33193 non-null  object
 6   Subject                    35000 non-null  object
 7   Mime-Version               34995 non-null  object
 8   Content-Type               34995 non-null  object
 9   Content-Transfer-Encoding  34995 non-null  object
 10  X-From                     34995 non-null  object
 11  X-To                       34995 non-null  object
 12  X-cc                       34995 non-null  object
 13  X-bcc                      34995 non-null  object
 14  X-Fold

In [ ]:
email_parsed = df.copy()

In [ ]:
def content_to_wordlist( content, remove_stopwords=False ):
    content = re.sub("[^a-zA-Z]"," ", content)
    words = content.lower().split()
    
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]

    return ' '.join(words)

In [ ]:
# enron_parsed['To'] = enron_parsed['To'].astype(str) # in case we want to use 'To' as information
data = pd.DataFrame(list(map(content_to_wordlist, 
                          email_parsed[['Subject', 'body']].apply(lambda x: ' '.join(x), axis=1))), 
                          columns = ["body"])

In [ ]:
data = data.assign(sender=email_sent["sender"].values)
data = data.replace({'sender': mapping})
data.head()

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df['body'])

In [ ]:
print(vectorizer.get_feature_names_out())

['00' '000' '0000' ... 'zznfvo' 'zztmky'
 'zzwyriqeuzgdapy59k830iuldikj7cgbt6v6dpd4720dkomzivepjdkb1rf6sdzeabikjctikmk0']


In [ ]:
X.toarray()

In [ ]:
#creat an array list for the senders
names = []
for name in df['user']:
    if name not in names:
        names.append(name)

# get the categories
# each sender is consider as an uniqle class
print("We have {} senders".format(len(names)))
# define the training set
# the first #% of body texts are consider as the training set
print("We have {} training samples".format(len(df['body']))) # to be changed...
# define the test set
# rest of the body texts are consider as the testing set
print("We have {} test samples".format(len(df['body']))) # to be changed...